In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

## Softmax

$ P(class = i) = \frac{e^i}{\sum e^i} $

In [3]:
z = torch.FloatTensor([1, 2, 3])

In [4]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [5]:
hypothesis.sum()

tensor(1.)

## Cross Entropy: 두 개의 확률분포가 얼마나 비슷한지

$ H(P, Q) = -\mathbb{E}_{x \sim P(x)}[\log Q(x)] = - \sum_{x \epsilon X} P(x) \log Q(x) $

### cross Entropy Loss:
$ L = \frac{1}{N} \sum -y \log \hat{y} $


In [6]:
z = torch.rand(3, 5, requires_grad=True)
print(z)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)  #prediction 예측값(y hat)  class: 5개 samples:3개

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)
tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [7]:
y = torch.randint(5, (3,))
print(y)  #이건 index값 원핫으로 바꿔주자
y.shape

tensor([0, 2, 1])


torch.Size([3])

In [8]:
y_one_hot = torch.zeros_like(hypothesis)
print(y.unsqueeze(1)) #1 dimension에 element 1을 넣어라
y_one_hot_1 = y_one_hot.scatter_(1, y.unsqueeze(1), 1)  #scatter(dimension=1, y unsqueeze해서, 1을 뿌림)
print(y_one_hot_1)
print(y_one_hot_1.shape)

tensor([[0],
        [2],
        [1]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])
torch.Size([3, 5])


In [9]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1)
print(cost)
print(cost.mean())

tensor([1.3301, 1.4602, 1.6165], grad_fn=<SumBackward2>)
tensor(1.4689, grad_fn=<MeanBackward0>)


## Cross-entropy Loss with torch.nn.functional

In [10]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [11]:
#high level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [12]:
#low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [14]:
#high level
F.nll_loss(F.log_softmax(z, dim=1), y)  #negative Log Likelihood

tensor(1.4689, grad_fn=<NllLossBackward>)

In [15]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

## Training with Low-level Cross Entroy Loss

In [17]:
x_train = [[1, 2, 1, 1], 
           [2, 1, 3, 2], 
           [3, 1, 3, 4], 
           [4, 1, 5, 5], 
           [1, 7, 5, 5], 
           [1, 2, 5, 6], 
           [1, 6, 6, 6], 
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

# samples = m , classes = 3, dim = 4

In [25]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True) # dim 4 -> 3 으로 가는 linear
b = torch.zeros(1, requires_grad=True)

#optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs= 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## Training with F.cross_entropy

In [26]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True) # dim 4 -> 3 으로 가는 linear
b = torch.zeros(1, requires_grad=True)

#optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs= 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산 (1)
    z = x_train.matmul(W) +b
    cost = F.cross_entropy(z, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## High-level implemntation with nn.Module

In [27]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3) #ouput이 3!
    
    def forward(self, x):
        return self.linear(x)

In [28]:
model = SoftmaxClassifierModel()

In [29]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    #cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    #cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609259
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269521
Epoch 1000/1000 Cost: 0.241922


* 클래스 2개일 때: Binary Cross Entropy / Sigmoid 사용
* 클래스 3개 이상일 때: Cross Entropy / Softmax 사용